## LangChain 사용해서 지원사업 공고 파일(pdf, hwp, png)을 요약해보자!

In [1]:
from openai import OpenAI
from getpass import getpass
from tqdm import tqdm

from utils.get_text import get_hwp_text, get_pdf_text, get_image_text # pdf, hwp에서 텍스트를 추출하는 사용자 지정 함수 

- LangChain 모델이 참고할 비슷한 용어 사전
- 개발팀이 직접 정의
- (추후에 유사한 단어가 나오면 사전에 추가하는 기능도 구현할 예정)

In [13]:
# 정의한 비슷한 용어 사전
keyword_dictionary = {
                        '신청기간' : ['신청 기간', '접수 기간'],
                        '신청방법' : ['지원 대상', '사업 대상', '융자 대상'],
                        '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
                        '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
                        '제출 서류' : ['필요 서류', '서류 제출 목록'],
                        '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준'],
                        '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용'],
                        '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의'],
                        '기타' : ['기타']
                        }

#### model 구현 준비
- OpenAI 객체 생성
- 파일 텍스트 추출
- model이 참고할 기본 템플릿 작성

In [2]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [18]:
# 파일 경로 설정 (백슬래시 문제 해결을 위해 r"" 사용)
file_path = f"금융\[광주] 북구 2025년 소상공인 대출 지원사업(특례보증ㆍ포용금융) 공고.png"

# 지원사업명 저장 (지원경로를 분할하여 사용)
title = file_path.split('\\')[1].split('.')[0]

# 파일 타입 저장 (지원경로를 분할하여 사용)
file_type = file_path.split('\\')[1].split('.')[1]

# 지원사업 텍스트 추출 (PDF/HWP 처리 함수 필요)
if file_path.endswith(".pdf"):
    full_text = get_pdf_text(file_path)
elif file_path.endswith(".hwp"):
    full_text = get_hwp_text(file_path)
elif file_path.endswith(".png"):
    full_text = get_image_text(file_path)

print(full_text)

<>:2: SyntaxWarning: invalid escape sequence '\['
<>:2: SyntaxWarning: invalid escape sequence '\['
C:\Users\every\AppData\Local\Temp\ipykernel_11916\3066476994.py:2: SyntaxWarning: invalid escape sequence '\['
  file_path = f"금융\[광주] 북구 2025년 소상공인 대출 지원사업(특례보증ㆍ포용금융) 공고.png"


스스 수 수 수 수 스스

Gaol

북구 CI'S 3% SHS 지원 - 무담보, 무이자, 무보증료
료

91142! 대출이자(5.5% 01 및 보증료 전액 지원(시후 입금)
GED 북구 관내 임차 소상공인
※ 지원제외 : 자가 또는 무상임차인 경우, 20244 특례보증 대출 이용자, 보증제외업종
339) - 대출한도 :업체당 최대 2천만원 (관광사업자, 20204 이후 재창업자 3천만원 우대!
- 대출 및 상환 : 일시상환(1년) / 원금균등분할상환(5년|
급 시점 산출금리 5.5% 이내)
EEED 광주은행, 광주문화신협, 북구 새마을금고(11개|, 광주농협 전 지점
*북구 새마을금고(11개)
*

:광주북부, 광주중앙 동부, SS, 서방, SBF, we, 예향 정다운, SS, chet

자특례보증, 포용금융특별대출 KIBMSISSM

SK 광주광역시북구



# OCR TEST

In [19]:
import cv2
import pytesseract
from PIL import Image

# Tesseract 실행 파일 경로 (Windows에서만 설정 필요)
pytesseract.pytesseract.tesseract_cmd = "C:\\Users\\every\\Desktop\\Project\\team_project\\SeSAC_Final\\AI-for-Creating-Business-Plan-Drafts\\OCR\\tesseract.exe"

# 이미지 로드
image_path = file_path  # 분석할 이미지 파일 경로
image = cv2.imread(image_path)

# 이미지 전처리 (그레이스케일 + 이진화)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# OCR 실행 (언어 설정: 한국어+영어+중국어)
custom_config = r'--oem 3 --psm 6 -l kor+eng+chi_sim'
text = pytesseract.image_to_string(binary, config=custom_config)

# 결과 출력
print(text)

ow
&
부구 AAO
2025 = SSH Ths
=
~
v 신청기간 es                             25. 3.4.~ KiB AAA
2                             Ee         =      으
북구 CIES 3% 특례보증 지원 - 무담보, 무이자, 무보증료
ㅎ1년간 대출이자(5.5% 이내) 및 보증료 전맥 지원(ㅅ,후입금)
60500 북구 관내 임차 소상공인
※ 지뭔제외 : 자가 또는 무상임차인 SL, 2024년 특례보증 대출 이용자, 보중제외업종
65539) - 대출한도 : 업체당 최대 2천만원 (관광사업자. 2020년 이후재창업자 3천만원 우다}
- 대출 및 상환 : 일시상환(1년) / 원금균등분할상환(5년)
- 대출금리 : 금융기관별 Alita 시점 산출금리 5.5% 이내)
SEED PSH, 광주문화신협, 북구 새마을금고[11개, 광주농협 전 지점
*북구 새마을금고(11개)
: 광주북부, 광주중앙, 동부, SS, 서방, 북광주, WS 예향, 정다운, 중흥, 태봉
문의 취급은행 영업점 또는 광주신용보증재단 홈페이지(^/^^/^^/.9]5100ㅇ.0「.<「)
개      =          =                         So
HALAS DUS 포용금융 OFA 지원
ETE 북구 관내 저신용-저소득 소상공인
PETE) BEL MBSE! 대출상품의 이자 5.0% 지원(년간
*서민금융형 HESS
: 햇살론15|| 햇살론뱅크, 최저신용자특례보증, 포용금융특별대출, <]2새희망홀씨||
   최대 3,500만원 / 최장 10년 x 대출상품별 상이
   광주은행 전 지점
문의 | 광주은행 영업점, 서민금융진흥원 북광주통합지원센터[| 397)
XY xs
DS 광주광역씨북구



In [21]:
import cv2
import numpy as np
import pytesseract
from PIL import Image

# 이미지 로드
image_path = file_path
image = cv2.imread(image_path)

# 1. 그레이스케일 변환
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 2. 가우시안 블러 (노이즈 제거)
blur = cv2.GaussianBlur(gray, (5,5), 0)

# 3. 적응형 이진화 (명암 대비 최적화)
binary = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# 4. 열림 연산 (잡음 제거)
kernel = np.ones((2,2), np.uint8)
processed = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

# OCR 실행
custom_config = r'--oem 3 --psm 6 -l kor+eng+chi_sim'
text2 = pytesseract.image_to_string(processed, config=custom_config)

# 결과 출력
print(text2)

QOD DO LO SO SF SO aia)
불구 AAIZO      “ ~
Q        —        s
O&-;           7:          기 | 기
Z                          LG

@OiBes    wi) ) BEG-TS4an
북구 CIS 38 특례보즘 지원 - 무담보, 무미자, 무보증료
ㅋ1년간 CHEBOIZKS.5% 0140) 및 보증료 전액 지원(시후입금)
   북구 관내 임차 소상공인

※ 지원제외 : 자가 또는 PAINS! 경우, 2024년 튼례보중 대졸 이용자, 보중제외업중
0339 - 대접한도: 업체당최대 2천만원 (긴관사업자 2020변이후재장업자3전민원우며

- 대중 및 상환 : 일시상환(1년) / 원금균등분할상환(5년

- 대줌금리 : 금융기관별 상이(취3 시점 산줌3리5.5% 이내)
   광주은행, 광주문화신협; 북구 새마옴금고(1130 광주농협 전 지점

*복구 새마용금고(1177}                                                           il

; 광주복부, 광주중앙, 동보, SS, 서방, 보광주, 28, 예향, 정다운, Se, 태봉
저신용저소득 소삼곰인 포용금융 OHA 지원
   북구 관내 저신웅:저소득 소상공인
GEE BROW 서민금몽형 HEA] 이자 5.0% 지원(1년간)

“MORES HAYS

THA Sil, HAS, 최저신용자본러보증 포용금융브법대줄, OBIS!
GEEZ) 최대3,500만원 / 최장 10년 ※ 대줄상품병상이                           ~
   광주온행 전 지점

애족 광주광역시복구



In [42]:
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
from matplotlib import pyplot as plt

from getpass import getpass
 
import uuid
import json
import time
import cv2
import requests

In [41]:
def put_text(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_coverted)
 
    if platform.system() == 'Darwin':
        font = 'AppleGothic.ttf'
    elif platform.system() == 'Windows':
        font = 'malgun.ttf'
        
    image_font = ImageFont.truetype(font, font_size)
    font = ImageFont.load_default()
    draw = ImageDraw.Draw(image)
 
    draw.text((x, y), text, font=image_font, fill=color)
    
    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)
 
    return opencv_image

In [43]:
api_url = getpass('<YOUR_API_URL> : ') # NAVER CLOVA invoke url 입력
secret_key = getpass('<YOUR_SECRET_KEY> : ') # NAVER CLOVA key 입력

In [44]:
path = file_path
files = [('file', open(path,'rb'))]

In [45]:
request_json = {'images': [{'format': 'jpg',
                                'name': 'demo'
                               }],
                    'requestId': str(uuid.uuid4()),
                    'version': 'V2',
                    'timestamp': int(round(time.time() * 1000))
                   }
 
payload = {'message': json.dumps(request_json).encode('UTF-8')}
 
headers = {
  'X-OCR-SECRET': secret_key,
}
 
response = requests.request("POST", api_url, headers=headers, data=payload, files=files)
result = response.json()

In [51]:
img = cv2.imread(path)
roi_img = img.copy()

# 추출된 텍스트를 저장할 리스트
extracted_texts = []

for field in result['images'][0]['fields']:
    text3 = field['inferText']
    extracted_texts.append(text3)  # 리스트에 저장

    vertices_list = field['boundingPoly']['vertices']
    pts = [tuple(vertice.values()) for vertice in vertices_list]
    topLeft = [int(_) for _ in pts[0]]
    topRight = [int(_) for _ in pts[1]]
    bottomRight = [int(_) for _ in pts[2]]
    bottomLeft = [int(_) for _ in pts[3]]

    cv2.line(roi_img, topLeft, topRight, (0,255,0), 2)
    cv2.line(roi_img, topRight, bottomRight, (0,255,0), 2)
    cv2.line(roi_img, bottomRight, bottomLeft, (0,255,0), 2)
    cv2.line(roi_img, bottomLeft, topLeft, (0,255,0), 2)

    roi_img = put_text(roi_img, text3, topLeft[0], topLeft[1] - 10, font_size=30)

# 모든 텍스트를 한 번에 출력
print("\n".join(extracted_texts))

당신의
부
2025
북구
소상공인
대출
지금
바로
신청하세요!
신청기간
(공통)
25.
3.4.~
자금
소진
시
북구
디딤돌
3
특례보증
지원
-
무담보,
무이자,
무보증료
→
1년간
대출이자(5.5%
이내)
및
보증료
전액
지원(사후
입금)
지원대상
북구
관내
임차
소상공인
지원제외
:
자가
또는
무상임차인
경우,
2024년
특례보증
대출
이용자,
보증제외업종
대출조건
-
대출한도:
업체당
최대
2천만원
(관광사업자,
2020년
이후
재창업자
3천만원
우대)
-
대출
및
상환
:
일시상환(1년)
/
원금균등분할상환(5년)
-
대출금리
:
금융기관별
상이
(취급
시점
산출금리
5.5%
이내)
취급은행
광주은행,
광주문화신협,
북구
새마을금고(11개),
광주농협
전
지점
*북구
새마을금고(11개)
1
:
광주북부,
광주중앙,
동부,
동운,
서방,
북광주,
빛고을,
예향,
정다운,
중흥,
태봉
문의
취급은행
영업점
또는
광주신용보증재단
홈페이지(www.gjsirbo.or.kr)
저신용·저소득
소상공인
포용금융
이차보전
지원
지원대상
북구
관내
저신용·저소득
소상공인
지원내용
광주은행
서민금융형
대출상품의
이자
5.0%
지원(1년간)
*
서민금융형
대출상품
:
햇살론1
511,
햇살론뱅크,
최저신용자특례보증,
포용금융특별대출,
KJB새희망홀씨Ⅱ
대출조건
최대
3,500만원
/
최장
10년
대출상품별
상이
2
취급은행
광주은행 전
지점
문의
광주은행
영업점,
서민금융진흥원
북광주통합지원센터(1397)
광주광역시북구


In [58]:
result = " ".join(extracted_texts)
result

'당신의 부 2025 북구 소상공인 대출 지금 바로 신청하세요! 신청기간 (공통) 25. 3.4.~ 자금 소진 시 북구 디딤돌 3 특례보증 지원 - 무담보, 무이자, 무보증료 → 1년간 대출이자(5.5% 이내) 및 보증료 전액 지원(사후 입금) 지원대상 북구 관내 임차 소상공인 지원제외 : 자가 또는 무상임차인 경우, 2024년 특례보증 대출 이용자, 보증제외업종 대출조건 - 대출한도: 업체당 최대 2천만원 (관광사업자, 2020년 이후 재창업자 3천만원 우대) - 대출 및 상환 : 일시상환(1년) / 원금균등분할상환(5년) - 대출금리 : 금융기관별 상이 (취급 시점 산출금리 5.5% 이내) 취급은행 광주은행, 광주문화신협, 북구 새마을금고(11개), 광주농협 전 지점 *북구 새마을금고(11개) 1 : 광주북부, 광주중앙, 동부, 동운, 서방, 북광주, 빛고을, 예향, 정다운, 중흥, 태봉 문의 취급은행 영업점 또는 광주신용보증재단 홈페이지(www.gjsirbo.or.kr) 저신용·저소득 소상공인 포용금융 이차보전 지원 지원대상 북구 관내 저신용·저소득 소상공인 지원내용 광주은행 서민금융형 대출상품의 이자 5.0% 지원(1년간) * 서민금융형 대출상품 : 햇살론1 511, 햇살론뱅크, 최저신용자특례보증, 포용금융특별대출, KJB새희망홀씨Ⅱ 대출조건 최대 3,500만원 / 최장 10년 대출상품별 상이 2 취급은행 광주은행 전 지점 문의 광주은행 영업점, 서민금융진흥원 북광주통합지원센터(1397) 광주광역시북구'

In [59]:
def get_image_text(path):
    api_url = 'https://dqfp3rdhcw.apigw.ntruss.com/custom/v1/39549/6ec703e8b494511fd50f11c0df474b47bdc8717eb906e3b6c5bab61e56621fba/general' # NAVER CLOVA invoke url 입력
    secret_key = 'U290ZXdjRFpBWVhPcFFGVWplRVRJT0RBQk9FS1VBb0k=' # NAVER CLOVA key 입력

    path = file_path
    files = [('file', open(path,'rb'))]

    request_json = {'images': [{'format': 'jpg',
                                'name': 'demo'
                               }],
                    'requestId': str(uuid.uuid4()),
                    'version': 'V2',
                    'timestamp': int(round(time.time() * 1000))
                   }
 
    payload = {'message': json.dumps(request_json).encode('UTF-8')}
    
    headers = {
    'X-OCR-SECRET': secret_key,
    }
    
    response = requests.request("POST", api_url, headers=headers, data=payload, files=files)
    result = response.json()

    img = cv2.imread(path)
    roi_img = img.copy()

    # 추출된 텍스트를 저장할 리스트
    extracted_texts = []

    for field in result['images'][0]['fields']:
        text = field['inferText']
        extracted_texts.append(text)  # 리스트에 저장

        vertices_list = field['boundingPoly']['vertices']
        pts = [tuple(vertice.values()) for vertice in vertices_list]
        topLeft = [int(_) for _ in pts[0]]
        topRight = [int(_) for _ in pts[1]]
        bottomRight = [int(_) for _ in pts[2]]
        bottomLeft = [int(_) for _ in pts[3]]

        cv2.line(roi_img, topLeft, topRight, (0,255,0), 2)
        cv2.line(roi_img, topRight, bottomRight, (0,255,0), 2)
        cv2.line(roi_img, bottomRight, bottomLeft, (0,255,0), 2)
        cv2.line(roi_img, bottomLeft, topLeft, (0,255,0), 2)

        roi_img = put_text(roi_img, text3, topLeft[0], topLeft[1] - 10, font_size=30)
  
    # 각 단어를 공백 구분자로 구분
    result = " ".join(extracted_texts)

    return result

In [60]:
print(get_image_text(file_path))

당신의 부 2025 북구 소상공인 대출 지금 바로 신청하세요! 신청기간 (공통) 25. 3.4.~ 자금 소진 시 북구 디딤돌 3 특례보증 지원 - 무담보, 무이자, 무보증료 → 1년간 대출이자(5.5% 이내) 및 보증료 전액 지원(사후 입금) 지원대상 북구 관내 임차 소상공인 지원제외 : 자가 또는 무상임차인 경우, 2024년 특례보증 대출 이용자, 보증제외업종 대출조건 - 대출한도: 업체당 최대 2천만원 (관광사업자, 2020년 이후 재창업자 3천만원 우대) - 대출 및 상환 : 일시상환(1년) / 원금균등분할상환(5년) - 대출금리 : 금융기관별 상이 (취급 시점 산출금리 5.5% 이내) 취급은행 광주은행, 광주문화신협, 북구 새마을금고(11개), 광주농협 전 지점 *북구 새마을금고(11개) 1 : 광주북부, 광주중앙, 동부, 동운, 서방, 북광주, 빛고을, 예향, 정다운, 중흥, 태봉 문의 취급은행 영업점 또는 광주신용보증재단 홈페이지(www.gjsirbo.or.kr) 저신용·저소득 소상공인 포용금융 이차보전 지원 지원대상 북구 관내 저신용·저소득 소상공인 지원내용 광주은행 서민금융형 대출상품의 이자 5.0% 지원(1년간) * 서민금융형 대출상품 : 햇살론1 511, 햇살론뱅크, 최저신용자특례보증, 포용금융특별대출, KJB새희망홀씨Ⅱ 대출조건 최대 3,500만원 / 최장 10년 대출상품별 상이 2 취급은행 광주은행 전 지점 문의 광주은행 영업점, 서민금융진흥원 북광주통합지원센터(1397) 광주광역시북구


- 템플릿 생성

In [56]:
base_template = {f'''
📌 **[{title}] 요약 안내**

📅 **신청기간**  

📝 **신청방법**  

🚫 **지원제외 대상**   

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  

📞 **문의처**  

📢 **기타 안내**  
'''
}

#### LangChain 모델로 간단하게 요약해보기
- (고도화 예정)

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
당신은 파일 요약 전문가입니다. 사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 요약해야합니다.
파일에 없는 내용은 공백으로 남겨두십시오.
요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니, 사전에 해당하는 단어가 나오면, 참고하여 요약해주십시오.

제공하는 템플릿 양식 : {base_template}
단어 정의 사전 : {keyword_dictionary}

---------------------------------------------------
사용자 요구 파일 : {text}

"""

# 파일 요약을 위한 프롬프트 템플릿을 정의
prompt = PromptTemplate.from_template(template)

# OpenAI의 GPT-4o 모델을 사용하여 채팅 모델을 생성
model = ChatOpenAI(
    model='gpt-4o',  # 사용할 언어 모델 지정
    api_key=MY_API_KEY  # OpenAI API 키 입력
)

# 모델의 출력을 문자열로 변환하는 출력 파서
output_parser = StrOutputParser()

# 체인(Chain) 구성: 프롬프트 → 모델 → 출력 파서 순서로 연결
chain = prompt | model | output_parser

In [57]:
print(chain.invoke({'base_template':base_template,
              'keyword_dictionary':keyword_dictionary,
              'text':text3}))

📌 **[[광주] 북구 2025년 소상공인 대출 지원사업(특례보증ㆍ포용금융) 공고] 요약 안내**

📅 **신청기간**  
- 2025년 3월 4일부터 자금 소진 시까지 

📝 **신청방법**  
- 북구 관내 임차 소상공인  
- 북구 관내 저신용·저소득 소상공인

🚫 **지원제외 대상**  
- 자가 또는 무상임차인, 2024년 특례보증 대출 이용자, 보증제외업종

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  
- 북구 디딤돌 3 특례보증 지원: 무담보, 무이자, 무보증료, 1년간 대출이자(5.5% 이내) 및 보증료 전액 지원  
- 저신용·저소득 소상공인 포용금융 이차보전 지원: 광주은행 서민금융형 대출상품의 이자 5.0% 지원(1년간)

📞 **문의처**  
- 취급은행 영업점 또는 광주신용보증재단 홈페이지(www.gjsirbo.or.kr)  
- 광주은행 영업점, 서민금융진흥원 북광주통합지원센터(1397)

📢 **기타 안내**  
